<a href="https://colab.research.google.com/github/mralamdari/CV-Yolo/blob/main/YOLO_v3_ObjectDetection_TensorFlow_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import PIL
import struct # used to convert native Python data types such as strings and numbers into a string of bytes and vice versa
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers.merge import add, concatenate

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d aruchomu/data-for-yolo-v3-kernel
!unzip \*.zip && rm *.zip

 96% 257M/267M [00:09<00:00, 24.8MB/s]
100% 267M/267M [00:09<00:00, 29.0MB/s]
Archive:  data-for-yolo-v3-kernel.zip
  inflating: coco.names              
  inflating: detections.gif          
  inflating: dog.jpg                 
  inflating: futur.ttf               
  inflating: office.jpg              
  inflating: yolov3.weights          


In [ ]:
class Weights_Reader:


  def __init__(self, filename):

    with open(filename, 'rb') as w_file:
      major, = struct.unpack('i', w_file.read(4))
      minor, = struct.unpack('i', w_file.read(4))
      revision, = struct.unpack('i', w_file.read(4))

      if (major*10 + minor) >= 2 and major < 1000 and minor < 1000:
        w_file.read(8)
      else:
        w_file.read(4)
      
      transpose = (major>1000) or (minor>1000)
      binary_weights = w_file.read()
      
    self.offset = 0
    self.all_weights = np.frombuffer(binary_weights, dtype='float32')

  def byte_reader(self, size):
    self.offset +=  size
    return self.all_weights[self.offset - size: self.offset]

  
  def load_weights(self, model):

    for i in range(106): # 53*2 = 106 layers in total
      try: # if it is a Convolutional Layer
        conv_layer = model.get_layer(f'conv_{i}')
        print(f'Loading weights of Convolution Layer {i}')

        if i not in [81, 93, 105]: #if it is not a Detection Layer (82, 94, 106)
         
          norm_layer = model.get_layer(f'bnorm_{i}')
          size = np.prod(norm_layer.get_weights()[0].shape) 

          bias = self.byte_reader(size)
          scale = self.byte_reader(size)
          mean = self.byte_reader(size)
          variance = self.byte_reader(size)

          weights = norm_layer.set_weights([scale, bias, mean, variance])

        if len(conv_layer.get_weights()) > 1:
          bias   = self.byte_reader(np.prod(conv_layer.get_weights()[1].shape))
          kernel = self.byte_reader(np.prod(conv_layer.get_weights()[0].shape))
          kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
          kernel = kernel.transpose([2, 3, 1, 0])
          conv_layer.set_weights([kernel, bias])
        else:
          kernel = self.byte_reader(np.prod(conv_layer.get_weights()[0].shape))
          kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
          kernel = kernel.transpose([2, 3, 1, 0])
          conv_layer.set_weights([kernel])

      except ValueError:
        print(f"Layer {i} is Not Convolution Layer")

        
  def reset(self):
    self.offset = 0   

In [ ]:
def conv_block(inputs, convolutions, skip=True):

  X = inputs
  count = 0
  conv_layers_count = len(convolutions)

  for conv_layer in convolutions:
    if count == conv_layers_count-2 and skip:
      skip_connection = X
    
    count += 1
    if convolutions['stride'] > 1:
      X = tf.keras.layers.ZeroPadding2d(((1, 0), (1, 0)))(X) # peculiar padding as darknet prefers left and top
    
    X = tf.keras.layers.Conv2D(filters = convolutions['filters'],
                               kernel  = convolutions['kernel'],
                               strides = convolutions['stride'],
                               padding = 'valid' if convolutions['stride']>1 else 'same',
                               use_bias= False if convolutions['bnorm'] else True,
                               name = f'conv_{convolutions['layer_idx']}')(X)
  
    if convolutions['bnorm']:
      X = tf.keras.layers.BatchNormalization(epsilon = 0.001, name=f"batch_{convolutions['layer_idx']}")(X)
    
    if convolutions['leaky']:
      X = tf.keras.layers.LeakyReLU(alpha=0.1, name=f"leaky_{convolutions['layer_idx']}")(X)
    
    return add([skip_connection, X]) if skip else X






def conv_block(inputs, convolutions, skip=True):

    X = inputs
    count = 0
    total_convolutions = len(convolutions)
    
    for conv in convolutions:

        if skip and count == (total_convolutions - 2):
            skip_connection = X

        count += 1
        if conv['stride'] > 1 :  
          X = tf.keras.layers.ZeroPadding2D(((1,0),(1,0)))(X) # peculiar padding as darknet prefers left and top


        X = tf.keras.layers.Conv2D(filters = conv['filter'],
                                    kernel_size = conv['kernel'],
                                    strides = conv['stride'],
                                    padding = 'valid' if conv['stride'] > 1 else 'same', # peculiar padding as darknet prefers left and top
                                    use_bias = False if conv['bnorm'] else True,
                                    name = f"conv_{conv['layer_idx']}")(X)


        if conv['bnorm']:
           X = tf.keras.layers.BatchNormalization(epsilon = 0.001, name = f"bnorm_{conv['layer_idx']}")(X)
        if conv['leaky']:
           X = tf.keras.layers.LeakyReLU(alpha = 0.1, name = f"leaky_{conv['layer_idx']}")(X)

    return add([skip_connection, X]) if skip else X


In [ ]:
def make_yolov3_model():

    input_image = tf.keras.layers.Input(shape=(None, None, 3))

    # Layers 0 to 4
    x = conv_block(input_image, [{'filter': 32, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 0},
                                {'filter': 64, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 1},
                                {'filter': 32, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 2},
                                {'filter': 64, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 3}])

    # Layers 5 to 8
    x = conv_block(x, [{'filter': 128, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 5},
                        {'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 6},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 7}])

    # Layers 9 to 11
    x = conv_block(x, [{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 9},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 10}])

    # Layers 12 to 15
    x = conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 12},
                        {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 13},
                        {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 14}])

    # Layers 16 to 36
    for i in range(7):
        x = conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 16+i*3},
                            {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 17+i*3}])
    skip_36 = x
    
    # Layers 37 to 40
    x = conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 37},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 38},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 39}])

    # Layers 41 to 61
    for i in range(7):
        x = conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 41+i*3},
                            {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 42+i*3}])
    skip_61 = x

    # Layers 62 to 65
    x = conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 62},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 63},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 64}])

    # Layers 66 to 74
    for i in range(3):
        x = conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 66+i*3},
                            {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 67+i*3}])

    # Layers 75 to 79
    x = conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 75},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 76},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 77},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 78},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 79}], skip=False)

    # Layers 80 to 82
    yolo_82 = conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 80},
                             {'filter':  255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)

    # Layers 83 to 86
    x = conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 84}], skip=False)
    x = tf.keras.layers.UpSampling2D(2)(x)
    x = concatenate([x, skip_61])

    # Layers 87 to 91
    x = conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 87},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 88},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 89},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 90},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 91}], skip=False)

    # Layers 92 to 94
    yolo_94 = conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 92},
                                                                                     {'filter': 255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 93}], skip=False)

    # Layers 95 to 98
    x = conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True,   'layer_idx': 96}], skip=False)
    x = tf.keras.layers.UpSampling2D(2)(x)
    x = concatenate([x, skip_36])

    # Layers 99 to 106
    yolo_106 = conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 99},
                                {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 100},
                                {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 101},
                                {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 102},
                                {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 103},
                                {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 104},
                                {'filter': 255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 105}], skip=False)

    model = tf.keras.models.Model(input_image, [yolo_82, yolo_94, yolo_106])
    return model